# 00-Connect database

In [67]:
import sqlite3
import pandas as pd
from pathlib import Path

SQL_OUT = Path("../sql_outputs/takeaway/")
SQL_OUT.mkdir(parents=True, exist_ok=True)

conn = sqlite3.connect("../databases/takeaway.db")

In [ ]:
# Checking DB Connection is in place
# pd.read_sql_query("SELECT 1 AS ok", conn)

# 01-Price distribution of menu items
Bins for price will be created in Tableau

In [ ]:
sql = """
SELECT price, COUNT(*) 
FROM menuitems 
GROUP BY price 
ORDER BY price
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "01_price_distribution.csv",
    index=False
)

df_result

# 02-Resto distribution per location

In [ ]:
sql = """
SELECT city, COUNT(*) as num_restaurants 
FROM restaurants 
GROUP BY city
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "02_resto_distribution_by_location.csv",
    index=False
)

df_result

# 03-Top 10 rated pizza
Top # will be handled in Tableau

In [69]:
sql = """
SELECT r.name, r.ratings, r.ratingsNumber
from restaurants as r
WHERE EXISTS (
    SELECT 1
    FROM categories c
    WHERE c.restaurant_id = r.primarySlug
      AND lower(c.name) LIKE '%pizza%'
)
ORDER BY r.ratings desc,r.ratingsNumber DESC
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "03_top_10_rated_pizza.csv",
    index=False
)

df_result

,name,ratings,ratingsNumber
0,De Echte Eethuis Carlos,5.0,599
1,Kebap - Pizza De Brug,5.0,312
2,Trend Kebab & Pizzeria,5.0,165
3,Munchies Ieper,5.0,6
4,La Trinacria Bottega,5.0,3
...,...,...,...
1344,Snack Muide,0.0,0
1345,De Nacht Wacht - Dranken & Snacks,0.0,0
1346,Mio Pasta Leuven,0.0,0
1347,Connect24-7 API Testshop,0.0,0


# 04-Map locations offering kapsalons and their average price

In [ ]:
sql = """
SELECT 
    r.name AS restaurant_name,
    r.latitude,
    r.longitude,
    AVG(m.price) AS avg_price,
    r.ratings
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE LOWER(m.name) LIKE '%kapsalon%'
GROUP BY r.name, r.latitude, r.longitude, r.ratings
ORDER BY avg_price DESC
"""

df_result = pd.read_sql_query(sql, conn)

# Save to CSV
df_result.to_csv(
    SQL_OUT / "04_kapsalon_location_avg_price.csv",
    index=False
)

df_result


# 05-Which restaurants have the best price-to-rating ratio?
Leaving some calculation for Tableau, like avg(m.price)/r.rating
Top # will be handled in Tableau

In [ ]:
sql = """
SELECT 
    r.name,
    r.ratings,
    AVG(m.price) AS avg_price
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE r.ratings > 0
GROUP BY r.name, r.ratings
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "05_best_price_to_rating_ratio.csv",
    index=False
)

df_result

# 06-Where are the delivery ‘dead zones’—areas with minimal restaurant coverage?
Just select all restos wtih locations now and use visualization later to identify the delivery dead zones

In [ ]:
sql = """
SELECT 
    l.city,
    l.latitude,
    l.longitude,
    COUNT(ltr.restaurant_id) AS restaurant_count
FROM locations AS l
LEFT JOIN locations_to_restaurants AS ltr
    ON l.ID = ltr.location_id
GROUP BY l.city, l.latitude, l.longitude
ORDER BY restaurant_count ASC
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "06_delivery_dead_zones.csv",
    index=False
)

df_result

# 07-How does the availability of vegetarian and vegan dishes vary by area?

In [ ]:
sql = """
SELECT
    r.name,
    r.latitude,
    r.longitude,
    MAX(c.name LIKE '%Vegetarian%') AS has_vegetarian,
    MAX(c.name LIKE '%Vegan%') AS has_vegan
FROM restaurants AS r
JOIN categories AS c
    ON r.primarySlug = c.restaurant_id
GROUP BY r.name, r.latitude, r.longitude
HAVING MAX(c.name LIKE '%Vegetarian%') + MAX(c.name LIKE '%Vegan%') > 0
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "07_veggie_vegan_by_area.csv",
    index=False
)

df_result

# 08-Identify the World Hummus Order (WHO): top 3 hummus serving restaurants
Top # will be handled in Tableau

In [ ]:
sql = """
SELECT DISTINCT
    r.name,
    r.ratings
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE lower(m.name) LIKE '%hummus%'
ORDER BY r.ratings DESC
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "08_WHO_top_3_hummus.csv",
    index=False
)

df_result

# 09-Which restaurants in Leuven provide best balance between rating and price?
Also handling some further calculation in Tableau like before for avg(m.price)/r.rating

In [ ]:
sql = """
SELECT 
    r.name,
    r.address,
    AVG(m.price) AS avg_price,
    r.ratings
FROM restaurants AS r
JOIN menuitems AS m
    ON r.primarySlug = m.primarySlug
WHERE r.city = 'Leuven' or r.city = 'Louvain'
GROUP BY r.name, r.address, r.ratings
HAVING r.ratings > 0
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "09_best_Leuven_price_rating.csv",
    index=False
)

df_result

# 10-Any restaurants with explicit keto or low carb menu items?

In [ ]:
sql = """
SELECT DISTINCT
    r.name AS restaurant_name,
    r.city
FROM menuitems AS m
JOIN restaurants AS r
    ON r.primarySlug = m.primarySlug
WHERE lower(m.name) LIKE '%keto%'
   OR lower(m.name) LIKE '%low carb%'
   OR lower(m.description) LIKE '%keto%'
   OR lower(m.description) LIKE '%low carb%'
"""
df_result = pd.read_sql_query(sql, conn)

df_result.to_csv(
    SQL_OUT / "10_restaurants_with_low_carb_options.csv",
    index=False
)

df_result